In [4]:
import torch.optim as optim
import torch
import torch.nn as nn
from tqdm import tqdm
import json
import numpy as np
import open3d as o3d
import sys
from copy import deepcopy
sys.path.append('Pointnet_Pointnet2_pytorch')
sys.path.append('Pointnet_Pointnet2_pytorch/models')


In [5]:
from scipy.spatial.transform import Rotation as R

def create_obb_from_description(centroid, dimensions, rotations):
    # Extract centroid, dimensions, and rotations
    cx, cy, cz = centroid
    length, width, height = dimensions
    rx, ry, rz = rotations * 360

    # Create the 8 corners of the box before rotation and translation
    dx = length / 2
    dy = width / 2
    dz = height / 2

    corners = np.array([
        [-dx, -dy, -dz],
        [ dx, -dy, -dz],
        [ dx,  dy, -dz],
        [-dx,  dy, -dz],
        [-dx, -dy,  dz],
        [ dx, -dy,  dz],
        [ dx,  dy,  dz],
        [-dx,  dy,  dz]
    ])

    # Apply rotations
    rotation = R.from_euler('xyz', [rx, ry, rz], degrees=True)
    rotated_corners = rotation.apply(corners)

    # Apply translation (centroid)
    translated_corners = rotated_corners + np.array([cx, cy, cz])

    return translated_corners

In [6]:
def generate_random_rotation_matrix():
    """Generate a random 3D rotation matrix."""
    angle_x = np.random.uniform(0, 2 * np.pi)
    angle_y = np.random.uniform(0, 2 * np.pi)
    angle_z = np.random.uniform(0, 2 * np.pi)

    Rx = np.array(
        [
            [1, 0, 0],
            [0, np.cos(angle_x), -np.sin(angle_x)],
            [0, np.sin(angle_x), np.cos(angle_x)],
        ]
    )

    Ry = np.array(
        [
            [np.cos(angle_y), 0, np.sin(angle_y)],
            [0, 1, 0],
            [-np.sin(angle_y), 0, np.cos(angle_y)],
        ]
    )

    Rz = np.array(
        [
            [np.cos(angle_z), -np.sin(angle_z), 0],
            [np.sin(angle_z), np.cos(angle_z), 0],
            [0, 0, 1],
        ]
    )

    R = Rz @ Ry @ Rx  # Combined rotation matrix
    return R


def apply_rotation(point_cloud, bounding_box, rotation_matrix):
    """Apply rotation to a point cloud and bounding box.
    ---------------
    Params:
    - point_cloud: Nx3 numpy array
    - bounding_box: tuple containing the bounding box parameters:
        - cx, cy, cz: center of the bounding box
        - sidex, sidey, sidez: dimensions of the bounding box
        - sin_rx, cos_rx, sin_ry, cos_ry, sin_rz, cos_rz: rotation parameters
    """
    # Rotate the point cloud
    rotated_point_cloud = deepcopy(point_cloud) @ rotation_matrix.T

    # Extract bounding box parameters
    cx, cy, cz, sidex, sidey, sidez, sin_rx, cos_rx, sin_ry, cos_ry, sin_rz, cos_rz = (
        deepcopy(bounding_box)
    )

    # Rotate the center of the bounding box
    bbox_center = np.array([cx, cy, cz])
    rotated_bbox_center = bbox_center @ rotation_matrix.T

    # Define the initial rotation matrix for the bounding box using the sine and cosine values
    initial_rotation_matrix = np.array(
        [
            [
                cos_ry * cos_rz,
                cos_rz * sin_rx * sin_ry - cos_rx * sin_rz,
                cos_rx * cos_rz * sin_ry + sin_rx * sin_rz,
            ],
            [
                cos_ry * sin_rz,
                cos_rx * cos_rz + sin_rx * sin_ry * sin_rz,
                -cos_rz * sin_rx + cos_rx * sin_ry * sin_rz,
            ],
            [-sin_ry, cos_ry * sin_rx, cos_rx * cos_ry],
        ]
    )

    # Combine the rotations: first the initial rotation, then the new rotation
    combined_rotation_matrix = rotation_matrix @ initial_rotation_matrix

    # Extract the new sine and cosine values from the combined rotation matrix
    sin_rx = np.sin(
        np.arctan2(combined_rotation_matrix[2, 1], combined_rotation_matrix[2, 2])
    )
    cos_rx = np.cos(
        np.arctan2(combined_rotation_matrix[2, 1], combined_rotation_matrix[2, 2])
    )
    sin_ry = np.sin(
        np.arctan2(
            -combined_rotation_matrix[2, 0],
            np.sqrt(
                combined_rotation_matrix[2, 1] ** 2
                + combined_rotation_matrix[2, 2] ** 2
            ),
        )
    )
    cos_ry = np.cos(
        np.arctan2(
            -combined_rotation_matrix[2, 0],
            np.sqrt(
                combined_rotation_matrix[2, 1] ** 2
                + combined_rotation_matrix[2, 2] ** 2
            ),
        )
    )
    sin_rz = np.sin(
        np.arctan2(combined_rotation_matrix[1, 0], combined_rotation_matrix[0, 0])
    )
    cos_rz = np.cos(
        np.arctan2(combined_rotation_matrix[1, 0], combined_rotation_matrix[0, 0])
    )

    # Create the rotated bounding box
    rotated_bounding_box = np.array([
        rotated_bbox_center[0],
        rotated_bbox_center[1],
        rotated_bbox_center[2],
        sidex,
        sidey,
        sidez,
        sin_rx,
        cos_rx,
        sin_ry,
        cos_ry,
        sin_rz,
        cos_rz,
    ])

    return rotated_point_cloud, rotated_bounding_box

In [7]:
def sincos_to_angle(sin_val, cos_val):
    angle = torch.atan2(sin_val, cos_val)  # Get the angle in radians
    angle = angle % (2 * np.pi)  # Ensure the angle is in the range [0, 2*pi)
    angle = angle / (2 * np.pi)  # Scale to [0, 1]
    return angle.numpy()

def angles_to_sincos(angles):
    sin_angles = torch.sin(angles * 2 * np.pi)  # Assuming angles are scaled between 0 and 1
    cos_angles = torch.cos(angles * 2 * np.pi)
    return np.array([sin_angles.numpy(), cos_angles.numpy()])

def normalize_point_cloud_and_obb(point_cloud, obb):
    """
    Normalize the point cloud and the relative oriented bounding box (OBB).
    
    Parameters:
    point_cloud (np.ndarray): The point cloud as an array of shape (N, 3).
    obb_vertices (np.ndarray): OBB is centroid, dimensions, and rotations. (3, 3, 3)
    
    Returns:
    np.ndarray: The normalized point cloud.
    np.ndarray: The normalized OBB, rotations are scaled between 0 and 1
    """
    # Calculate the centroid of the point cloud
    centroid = np.mean(point_cloud, axis=0)
    obb_centroid = obb[:3]
    obb_dimensions = obb[3:6]
    obb_rotations = obb[6:]
    # Center the OBB vertices and point cloud points at the origin
    
    centered_pc = point_cloud - centroid
    obb_centroid = obb_centroid - centroid
    # Calculate the maximum distance from the origin for the point cloud points
    max_distance = np.max(np.linalg.norm(centered_pc, axis=1))
    
    # Scale the OBB vertices and point cloud points
    scaled_pc = centered_pc / max_distance
    scaled_obb_centroid = obb_centroid / max_distance
    scaled_obb_dimensions = obb_dimensions / max_distance
    scaled_obb_rotations = obb_rotations / 360

    sin_cos_rx = angles_to_sincos(torch.tensor(scaled_obb_rotations[0]))
    sin_cos_ry = angles_to_sincos(torch.tensor(scaled_obb_rotations[1]))
    sin_cos_rz = angles_to_sincos(torch.tensor(scaled_obb_rotations[2]))
    scaled_obb_rotations = np.concatenate([sin_cos_rx, sin_cos_ry, sin_cos_rz])
    scaled_obb = np.concatenate([scaled_obb_centroid, scaled_obb_dimensions, scaled_obb_rotations])

    
    return scaled_pc, scaled_obb, max_distance, centroid

def denormalize_point_cloud_and_obb(normalized_pc, normalized_obb, max_distance, original_centroid):
    """
    Denormalize the point cloud and the relative oriented bounding box (OBB).
    
    Parameters:
    normalized_pc (np.ndarray): The normalized point cloud as an array of shape (N, 3).
    normalized_obb (np.ndarray): The normalized OBB, rotations are scaled between 0 and 1.
    max_distance (float): The maximum distance used for normalization.
    original_centroid (np.ndarray): The original centroid of the point cloud.
    
    Returns:
    np.ndarray: The denormalized point cloud.
    np.ndarray: The denormalized OBB.
    """
    # Extract normalized OBB components
    normalized_obb_centroid = deepcopy(normalized_obb[:3])
    normalized_obb_dimensions = deepcopy(normalized_obb[3:6])
    
    sin_cos_rx = torch.tensor(deepcopy(normalized_obb[6:8]))
    sin_cos_ry = torch.tensor(deepcopy(normalized_obb[8:10]))
    sin_cos_rz = torch.tensor(deepcopy(normalized_obb[10:12]))
    
    # Convert sin and cos back to angles
    denorm_rx = sincos_to_angle(sin_cos_rx[0], sin_cos_rx[1])
    denorm_ry = sincos_to_angle(sin_cos_ry[0], sin_cos_ry[1])
    denorm_rz = sincos_to_angle(sin_cos_rz[0], sin_cos_rz[1])
    
    denorm_rotations = np.array([denorm_rx, denorm_ry, denorm_rz]) #* 360
    
    # Denormalize the point cloud and OBB components
    denorm_pc = normalized_pc * max_distance + original_centroid
    denorm_obb_centroid = normalized_obb_centroid * max_distance + original_centroid
    denorm_obb_dimensions = normalized_obb_dimensions * max_distance
    
    denorm_obb = np.concatenate([denorm_obb_centroid, denorm_obb_dimensions, denorm_rotations])
    
    return denorm_pc, denorm_obb

In [8]:
dataset = json.load(open('gt_dataset2.json'))
for obj in dataset:
    dataset[obj]['points'] = np.array(dataset[obj]['points']).reshape(-1, 3)
    dataset[obj]['box'] = np.array(dataset[obj]['box'])

In [9]:
#normalize input and output data
min_rot, max_rot = np.inf, -np.inf
for obj in dataset.values():
   rotations = obj['box'][6:]
   obj['points'], obj['box'], obj['max_distance'], obj['centroid'] = normalize_point_cloud_and_obb(obj['points'], obj['box'])


In [10]:
index = int(list(dataset.keys())[-1]) + 1
to_append = {}
for obj in dataset.values():
    for _ in range(3):
        rotated_pc, rotated_box = apply_rotation(obj['points'], obj['box'], generate_random_rotation_matrix())
        max_distance = max_distance = np.max(np.linalg.norm(rotated_pc, axis=1))
        centroid = np.mean(rotated_pc, axis=0)
        to_append[index] = {'points': rotated_pc, 'box': rotated_box, 'max_distance': max_distance, 'centroid': centroid}
        index += 1


In [11]:
dataset.update(to_append)
len(dataset)

408

In [12]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = list(dataset.values())
        self.maxpoints = max([len(obj['points']) for obj in self.dataset])
        self.centroid = [obj['centroid'] for obj in self.dataset]
        self.max_distance = [obj['max_distance'] for obj in self.dataset]

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        points = torch.tensor(self.dataset[idx]['points'], dtype = torch.float32)
        if points.shape[0] > 1024:
            points = points[torch.randperm(points.shape[0])[:1024]]
        if points.shape[0] < 1024:
            points = torch.cat([points, torch.zeros(1024 - len(points), 3)])
        assert points.shape[0] == 1024
        
        #points = torch.cat([points, torch.zeros(self.maxpoints - len(points), 3)])
        points = points.permute(1, 0)
        box = torch.tensor(self.dataset[idx]['box'], dtype = torch.float32).unsqueeze(0)
        return points, box, self.max_distance[idx], self.centroid[idx]

In [13]:
def draw(points, box):
    centroid = box[0][:3]
    dimensions = box[0][3:6]
    rotations = box[0][6:]
    rx = sincos_to_angle(rotations[0], rotations[1])
    ry = sincos_to_angle(rotations[2], rotations[3])
    rz = sincos_to_angle(rotations[4], rotations[5])
    box = create_obb_from_description(centroid, dimensions, np.array([rx, ry, rz]))
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points.permute(1, 0).numpy())
    box = o3d.geometry.OrientedBoundingBox.create_from_points(o3d.utility.Vector3dVector(box))
    box.color = (1, 0, 0)
    o3d.visualization.draw_geometries([pcd, box])

In [14]:
#draw(*Dataset(dataset)[20][:2])

In [15]:
from sklearn.model_selection import train_test_split
data = Dataset(dataset)
train_data, test_data = train_test_split(data, test_size=0.2)
len(train_data), len(test_data)

(326, 82)

In [16]:
idx = 201
sample = data[idx]
pc = sample[0]
box = sample[1]
max_distance = sample[2]
centroid = sample[3]

In [17]:
pc = pc.permute(1, 0).numpy()
box = box.numpy()

In [18]:
rotated_pc, rotated_box = apply_rotation(pc, box[0], generate_random_rotation_matrix())

In [19]:
rotated_pc = torch.tensor(rotated_pc).permute(1, 0)
rotated_box = torch.tensor(rotated_box).unsqueeze(0)
rotated_pc.shape, rotated_box.shape

(torch.Size([3, 1024]), torch.Size([1, 12]))

In [20]:
draw(data[idx][0], data[idx][1])

In [21]:
draw(rotated_pc, rotated_box)

In [22]:
denormalized_pc, denormalized_box = denormalize_point_cloud_and_obb(pc.permute(1, 0).numpy(), box[0].numpy(), max_distance, centroid)

AttributeError: 'numpy.ndarray' object has no attribute 'permute'

In [65]:
denormalized_box, box

(array([-0.09201974,  0.13136518, -0.16020407,  0.80898261,  0.77780479,
         1.35952616,  0.06569515,  0.89122862,  0.24758725]),
 tensor([[-0.0920,  0.1314, -0.1602,  0.8090,  0.7778,  1.3595,  0.4012,  0.9160,
          -0.6315,  0.7754,  0.9999,  0.0152]]))

In [66]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(denormalized_pc)
pcd.paint_uniform_color([1, 0, 0])
centroid, dimensions, rotations = (
    deepcopy(denormalized_box[:3]),
    deepcopy(denormalized_box[3:6]),
    deepcopy(denormalized_box[6:]),
)
box = create_obb_from_description(centroid, dimensions, rotations)
box = o3d.geometry.OrientedBoundingBox.create_from_points(
    o3d.utility.Vector3dVector(box)
)
box.color = (1, 0, 0)

o3d.visualization.draw_geometries([pcd, box])

In [67]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=16, shuffle=True)

In [122]:
from Pointnet_Pointnet2_pytorch.models.pointnet_utils import PointNetEncoder
class PointCloudNet(nn.Module):
    def __init__(self):
        super(PointCloudNet, self).__init__()
        self.backbone = PointNetEncoder(global_feat=True, feature_transform=False, channel = 3)
        self.fc = nn.Sequential(
            nn.Linear(1024, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 12)
        )
    
    def forward(self, x):
        x = self.backbone(x)
        x = x[0]
        x = x.view(x.size(0), -1)  # Flatten the output for the fully connected layer
        x = self.fc(x)
        x = x.view(-1, 1, 12)
        return x

    def train_model(self, num_epochs, optimizer, criterion, train_loader, test_loader, device):
        best_val_loss = np.inf
        pbar = tqdm(range(num_epochs))
        for epoch in pbar:
            model.train()
            running_loss = 0.0
            for data in train_loader:
                points, box, _, _ = data                
                points, box = points.to(device), box.to(device)
                optimizer.zero_grad()
                outputs = model(points)
                loss = criterion(outputs, box)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            
            train_loss = running_loss / len(train_loader)

            model.eval()
            running_loss = 0.0
            for data in test_loader:
                points, box, _, _ = data
                points, box = points.to(device), box.to(device)
                outputs = model(points)
                loss = criterion(outputs, box)
                running_loss += loss.item()
            
            val_loss = running_loss / len(test_loader)
            
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(model.state_dict(), "pc_net.pth")
            
            pbar.set_postfix({'train_loss': train_loss, 'val_loss': val_loss, 'best_val_loss': best_val_loss})

device = torch.device("mps")
model = PointCloudNet()
model.to(device)
model.train()
x = torch.randn(10, 3, 120, device = device)
model(x).shape


torch.Size([10, 1, 12])

In [123]:
class EasyNet(nn.Module):
    def __init__(self):
        super(EasyNet, self).__init__()
        self.backbone = nn.Sequential(
            nn.Conv1d(3, 64, 1),
            nn.ReLU(),
            nn.Conv1d(64, 64, 1),
            nn.ReLU(),
            nn.Conv1d(64, 64, 1),
            nn.ReLU(),
            nn.Conv1d(64, 128, 1),
            nn.ReLU(),
            nn.Conv1d(128, 1024, 1),
            nn.AdaptiveAvgPool1d(1)
        )
        self.fc = nn.Sequential(
            nn.Linear(1024, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 12)
        )
    def forward(self, x):
        x = self.backbone(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = x.view(-1, 1, 12)
        return x
    def train_model(self, num_epochs, optimizer, criterion, train_loader, test_loader, device):
        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0
            for i, data in enumerate(train_loader):
                points, box, _, _ = data                
                points, box = points.to(device), box.to(device)
                optimizer.zero_grad()
                outputs = model(points)
                loss = criterion(outputs, box)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            
            if epoch % 10 == 0:
                print(f"Epoch {epoch} - Loss: {running_loss / len(train_loader)}")

                model.eval()
                running_loss = 0.0
                for i, data in enumerate(tqdm(test_loader)):
                    points, box, _, _ = data
                    points, box = points.to(device), box.to(device)
                    outputs = model(points)
                    loss = criterion(outputs, box)
                    running_loss += loss.item()
                print(f"Validation Loss: {running_loss / len(test_loader)}")

model = EasyNet()
model.to(device)
model.train()
x = torch.randn(10, 3, 1024, device = device)
model(x).shape

torch.Size([10, 1, 12])

In [124]:
from pointnet2_utils import PointNetSetAbstractionMsg, PointNetSetAbstraction
from Pointnet_Pointnet2_pytorch.models.pointnet2_cls_ssg import get_model

class Backbone(nn.Module):
    def __init__(self,normal_channel=True):
        super(Backbone, self).__init__()
        in_channel = 6 if normal_channel else 3
        self.normal_channel = normal_channel
        self.sa1 = PointNetSetAbstraction(npoint=512, radius=0.2, nsample=32, in_channel=in_channel, mlp=[64, 64, 128], group_all=False)
        self.sa2 = PointNetSetAbstraction(npoint=128, radius=0.4, nsample=64, in_channel=128 + 3, mlp=[128, 128, 256], group_all=False)
        self.sa3 = PointNetSetAbstraction(npoint=None, radius=None, nsample=None, in_channel=256 + 3, mlp=[256, 512, 1024], group_all=True)
    
    def forward(self, xyz):
        B, _, _ = xyz.shape
        if self.normal_channel:
            norm = xyz[:, 3:, :]
            xyz = xyz[:, :3, :]
        else:
            norm = None
        l1_xyz, l1_points = self.sa1(xyz, norm)
        l2_xyz, l2_points = self.sa2(l1_xyz, l1_points)
        l3_xyz, l3_points = self.sa3(l2_xyz, l2_points)
        x = l3_points.view(B, 1024)
        return x

def load_backbone():
    state_dict = "Pointnet_Pointnet2_pytorch/log/classification/pointnet2_ssg_wo_normals/checkpoints/best_model.pth"
    d = torch.load(state_dict, map_location='cpu')['model_state_dict']
    model = Backbone(normal_channel=False)
    for key in d:
        if key in model.state_dict():
            model.state_dict()[key] = d[key]

    return model

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.backbone = load_backbone()
        for name, param in self.backbone.named_parameters():
            param.requires_grad = False
        self.fc = nn.Linear(1024, 12)
    
    def forward(self, x):
        x = self.backbone(x)
        x = self.fc(x)
        x = x.view(-1, 1, 12)
        return x
    
    def train_model(self, num_epochs, optimizer, criterion, train_loader, test_loader, device):
        best_val_loss = np.inf
        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0
            for i, data in enumerate(tqdm(train_loader)):
                points, box, _, _ = data                
                points, box = points.to(device), box.to(device)
                optimizer.zero_grad()
                outputs = model(points)
                loss = criterion(outputs, box)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
            
            print(f"Epoch {epoch} - Loss: {running_loss / len(train_loader)}")

            model.eval()
            running_loss = 0.0
            for i, data in enumerate(tqdm(test_loader)):
                points, box, _, _ = data
                points, box = points.to(device), box.to(device)
                outputs = model(points)
                loss = criterion(outputs, box)
                running_loss += loss.item()
            
            if running_loss < best_val_loss:
                best_val_loss = running_loss
                torch.save(model.state_dict(), "best_model2.pth")
            print(f"Validation Loss: {running_loss / len(test_loader)}")

x = torch.randn(10, 3, 1024)
model = Model()
model(x).shape

torch.Size([10, 1, 12])

In [128]:
# Define the regression loss function
criterion = nn.MSELoss(reduction="sum")

# Initialize the model, optimizer, and criterion
model = PointCloudNet()
device = torch.device("mps")
optimizer = optim.Adam(model.parameters(), lr=0.005)
model = model.to(device)

# Example training loop
num_epochs = 200
model.train_model(num_epochs, optimizer, criterion, train_loader, test_loader, device)

100%|██████████| 200/200 [02:25<00:00,  1.38it/s, train_loss=67.1, val_loss=40.2, best_val_loss=39]  


In [129]:
model = PointCloudNet()
device = torch.device("mps")
model = model.to(device)
model.load_state_dict(torch.load("pc_net.pth"))

<All keys matched successfully>

In [173]:
def draw(points, gt_box, pred_box):
    centroid = deepcopy(gt_box[0][:3])
    dimensions = deepcopy(gt_box[0][3:6])
    rotations = deepcopy(gt_box[0][6:])
    rx = sincos_to_angle(rotations[0], rotations[1])
    ry = sincos_to_angle(rotations[2], rotations[3])
    rz = sincos_to_angle(rotations[4], rotations[5])
    gt_box = create_obb_from_description(centroid, dimensions, np.array([rx, ry, rz]))

    pred_box = deepcopy(pred_box[0])
    centroid = deepcopy(pred_box[:3])
    dimensions = deepcopy(pred_box[3:6])
    rotations = deepcopy(pred_box[6:])
    rx = sincos_to_angle(rotations[0], rotations[1])
    ry = sincos_to_angle(rotations[2], rotations[3])
    rz = sincos_to_angle(rotations[4], rotations[5])
    pred_box = create_obb_from_description(centroid, dimensions, np.array([rx, ry, rz]))

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    gt_box = o3d.geometry.OrientedBoundingBox.create_from_points(o3d.utility.Vector3dVector(gt_box))
    gt_box.color = (1, 0, 0)

    pred_box = o3d.geometry.OrientedBoundingBox.create_from_points(o3d.utility.Vector3dVector(pred_box))
    pred_box.color = (0, 1, 0)
    o3d.visualization.draw_geometries([pcd, gt_box, pred_box])

In [174]:
points, box, max_distance, centroid = next(iter(test_loader))
idx = 5
model.eval()
sample_pc = points[idx].permute(1, 0).numpy()
sample_box = box[idx]

pred_box = model(points.to(device))[idx].detach().cpu()

In [175]:
pred_box, sample_box

(tensor([[-5.3533e-03,  1.9682e-02, -2.0012e-02,  1.2397e+00,  8.5698e-01,
           1.4516e+00,  5.5624e-02,  2.2031e-01, -9.2282e-04,  7.7289e-01,
           1.4975e-01,  1.7810e-01]]),
 tensor([[ 0.0286, -0.2116, -0.0687,  1.0836,  0.6205,  1.6926, -0.5067,  0.8621,
           0.6622,  0.7493,  0.7698,  0.6383]]))

In [176]:
draw(sample_pc, sample_box, pred_box)

In [92]:
sample_box

tensor([[-0.0226,  0.3352, -0.0308,  1.2851,  0.1802,  0.6912,  0.0000,  1.0000,
          0.0133,  0.9999,  0.9980,  0.0629]])